**- Exercice 1 : Gestion des collisions par chaînage**

In [1]:
import random  # Pour générer des valeurs aléatoires
import timeit  # Pour mesurer le temps d'exécution

def creerTable(m):
    return [[] for _ in range(m)]  # Crée une table de hachage à chaînage avec m listes vides

def h(c, m):
    return c % m  # Fonction de hachage simple (modulo)

def inserer(t, c, m):
    t[h(c, m)].append(c)  # Ajoute la valeur c dans la chaîne correspondant à son indice de hachage

def rechercher(t, c, m):
    return c in t[h(c, m)]  # Recherche la valeur c dans la chaîne de son indice de hachage

def supprimer(t, c, m):
    if c in t[h(c, m)]:
        t[h(c, m)].remove(c)  # Supprime la valeur c si elle est présente dans la chaîne

def tempsChainage(m):
    n = 2 * m  # Nombre d'éléments à insérer (facteur de charge 2)
    t = creerTable(m)  # Crée la table de hachage
    valeurs = [random.randint(0, 10**9) for _ in range(n)]  # Génère n entiers aléatoires
    return timeit.timeit(lambda: [inserer(t, v, m) for v in valeurs], number=1)  # Mesure le temps d'insertion

tailles = [100, 1000, 10000, 100000]  # Différentes tailles de table à tester
temps_chaine = [tempsChainage(m) for m in tailles]  # Mesure les temps pour chaque taille
print(temps_chaine)  # Affiche les temps mesurés

[8.720000187167898e-05, 0.0008067000017035753, 0.016303900003549643, 0.11946399999578716]


**- Exercice 2 : Table de hachage avec adressage ouvert**

In [2]:
def tableVide(m):
    return [None] * m  # Crée une table vide de taille m (adressage ouvert)

def h1(c, m):
    return c % m  # Première fonction de hachage (modulo)

def h2(c, m):
    return 1 + (c % (m - 1))  # Deuxième fonction de hachage pour le double hachage

def insererLineaire(t, c, m):
    for i in range(m):
        j = (h1(c, m) + i) % m  # Recherche linéaire : avance de 1 à chaque collision
        if t[j] is None:
            t[j] = c  # Place la valeur c à la première case libre
            return

def insererQuadratique(t, c, m):
    for i in range(m):
        j = (h1(c, m) + i * i) % m  # Recherche quadratique : avance de i^2 à chaque collision
        if t[j] is None:
            t[j] = c  # Place la valeur c à la première case libre
            return

def insererDouble(t, c, m):
    for i in range(m):
        j = (h1(c, m) + i * h2(c, m)) % m  # Double hachage : avance d'un pas variable selon h2
        if t[j] is None:
            t[j] = c  # Place la valeur c à la première case libre
            return

def tempsLineaire(m):
    n = int(0.7 * m)  # Facteur de charge 0.7
    t = tableVide(m)  # Crée la table vide
    v = [random.randint(0, m - 1) for _ in range(n)]  # Génère n valeurs aléatoires
    return timeit.timeit(lambda: [insererLineaire(t, x, m) for x in v], number=1)  # Mesure le temps d'insertion

def tempsQuadratique(m):
    n = int(0.7 * m)
    t = tableVide(m)
    v = [random.randint(0, m - 1) for _ in range(n)]
    return timeit.timeit(lambda: [insererQuadratique(t, x, m) for x in v], number=1)

def tempsDouble(m):
    n = int(0.7 * m)
    t = tableVide(m)
    v = [random.randint(0, m - 1) for _ in range(n)]
    return timeit.timeit(lambda: [insererDouble(t, x, m) for x in v], number=1)

tailles = [11, 101, 1009, 10007]  # Différentes tailles de table à tester
temps_L = [tempsLineaire(m) for m in tailles]  # Temps pour l'insertion linéaire
temps_Q = [tempsQuadratique(m) for m in tailles]  # Temps pour l'insertion quadratique
temps_D = [tempsDouble(m) for m in tailles]  # Temps pour le double hachage
print(temps_L)  # Affiche les temps mesurés pour l'insertion linéaire
print(temps_Q)  # Affiche les temps mesurés pour l'insertion quadratique
print(temps_D)  # Affiche les temps mesurés pour le double hachage

[1.069999416358769e-05, 5.7199998991563916e-05, 0.0006500999952550046, 0.007118499997886829]
[1.1200005246791989e-05, 5.499999679159373e-05, 0.0006739000018569641, 0.0067086999988532625]
[1.7799997294787318e-05, 8.779999916441739e-05, 0.0009222000007866882, 0.009107900004892144]


**- Exercice 3 : Filtres de Bloom**

In [3]:
def hBloom(c, i, m):
    return (c * (i + 1) + (i + 3)) % m  # Fonction de hachage pour le filtre de Bloom, dépend de c, i et m

def creerBloom(m):
    return [0] * m  # Crée un filtre de Bloom vide de taille m

def ajouterBloom(bloom, c, m, k):
    for i in range(k):
        bloom[hBloom(c, i, m)] = 1  # Pour chaque fonction de hachage, place un 1 à la position calculée

def testBloom(bloom, c, m, k):
    for i in range(k):
        if bloom[hBloom(c, i, m)] == 0:  # Si une des positions est à 0, c n'est pas dans le filtre
            return False
    return True  # Si toutes les positions sont à 1, c peut être dans le filtre (ou faux positif)

def tauxFauxPositifs(n, m, k):
    b = creerBloom(m)  # Crée un filtre de Bloom vide
    vrais = [random.randint(0, 10**9) for _ in range(n)]  # Génère n valeurs à insérer
    for x in vrais:
        ajouterBloom(b, x, m, k)  # Ajoute chaque valeur dans le filtre
    faux = 0
    tests = 5000  # Nombre de tests pour estimer le taux de faux positifs
    for _ in range(tests):
        x = random.randint(0, 10**9)
        if x not in vrais and testBloom(b, x, m, k):  # Si x n'est pas dans vrais mais le filtre dit oui
            faux += 1
    return faux / tests  # Retourne le taux de faux positifs

n = 5000  # Nombre d'éléments à insérer dans le filtre
m = 25000  # Taille du filtre de Bloom
p2 = tauxFauxPositifs(n, m, 2)  # Taux de faux positifs avec 2 fonctions de hachage
p3 = tauxFauxPositifs(n, m, 3)  # Taux de faux positifs avec 3 fonctions de hachage
p4 = tauxFauxPositifs(n, m, 4)  # Taux de faux positifs avec 4 fonctions de hachage
print(p2, p3, p4)  # Affiche les taux de faux positifs obtenus

0.2248 0.2304 0.2846


**- Exercice 4 : Count-Min Sketch**

In [4]:
def hCMS(c, i, m):
    return (c * (i + 1) + 3 * i) % m  # Fonction de hachage pour la ligne i et la clé c

def creerCMS(d, m):
    return [[0] * m for _ in range(d)]  # Crée une matrice d x m initialisée à 0

def ajouterCMS(cms, c, d, m):
    for i in range(d):
        cms[i][hCMS(c, i, m)] += 1  # Incrémente la case correspondant à la clé c pour chaque ligne

def estimerCMS(cms, c, d, m):
    return min(cms[i][hCMS(c, i, m)] for i in range(d))  # Estime la fréquence de c (minimum sur les lignes)

def precisionCMS(d, m, n):
    cms = creerCMS(d, m)  # Crée le sketch vide
    vrais = []  # Liste des vraies valeurs insérées
    for _ in range(n):
        x = random.randint(0, 20000)
        vrais.append(x)
        ajouterCMS(cms, x, d, m)  # Ajoute x dans le sketch
    erreurs = 0
    tests = 5000  # Nombre de tests pour estimer la précision
    for _ in range(tests):
        x = random.randint(0, 20000)
        if estimerCMS(cms, x, d, m) > vrais.count(x):  # Si l'estimation surestime la vraie fréquence
            erreurs += 1
    return erreurs / tests  # Retourne le taux d'erreur (surestimation)

m = 200  # Nombre de colonnes (taille de chaque ligne)
n = 500  # Nombre de valeurs à insérer
p2 = precisionCMS(2, m, n)  # Précision avec 2 lignes
p3 = precisionCMS(3, m, n)  # Précision avec 3 lignes
p4 = precisionCMS(4, m, n)  # Précision avec 4 lignes
print(p2, p3, p4)  # Affiche les taux d'erreur pour chaque configuration

0.9254 0.9044 0.8916
 0.9044 0.8916
